In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
%matplotlib inline

In [3]:
# Adapte le chemin et le nom du fichier
df = pd.read_csv('../data/raw/passengers_casatramway_raw.csv')
print("Aperçu des données brutes :")
print(df.head())
print("\nInformations initiales :")
print(df.info())

Aperçu des données brutes :
         Date Jour_Semaine  Mois  Annee  Est_Week_End  Est_Jour_Ferie  \
0  2022-01-01     Saturday     1   2022             1               1   
1  2022-01-02       Sunday     1   2022             1               0   
2  2022-01-03       Monday     1   2022             0               0   
3  2022-01-04      Tuesday     1   2022             0               0   
4  2022-01-05    Wednesday     1   2022             0               0   

   Est_Vacances_Scolaires  Evenement_Special  Temperature_Moyenne_C  \
0                       0                  0                    4.8   
1                       0                  0                   10.2   
2                       0                  0                    3.0   
3                       0                  0                    4.5   
4                       0                  0                    5.5   

   Precipitations_mm  Nb_Passagers  
0                3.7         18493  
1                3.5         243

In [4]:
df['Date'] = pd.to_datetime(df['Date'])
df['Jour'] = df['Date'].dt.day
df['Numero_Semaine'] = df['Date'].dt.isocalendar().week.astype(int)
df['Jour_Annee'] = df['Date'].dt.dayofyear
df['Trimestre'] = df['Date'].dt.quarter
df['Jour_Mois'] = df['Date'].dt.day

In [5]:
df = df.drop(columns=['Date']) # Ou la garder si l'index est la date plus tard

In [6]:
categorical_features = ['Jour_Semaine']

# Créer un ColumnTransformer pour l'encodage OneHot
# 'remainder="passthrough"' garde les autres colonnes non transformées
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

# Appliquer le preprocessor
# Fit et transform sur le DataFrame
df_processed_temp = preprocessor.fit_transform(df)

# Récupérer les noms des colonnes après OneHotEncoding
onehot_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)

# Créer le nouveau DataFrame avec les colonnes encodées et les autres
# On récupère les colonnes qui n'ont pas été encodées
remaining_features = [col for col in df.columns if col not in categorical_features]

# Convertir df_processed_temp en DataFrame Pandas
df_processed = pd.DataFrame(df_processed_temp, columns=list(onehot_feature_names) + remaining_features)

In [7]:
target_column = 'Nb_Passagers'
features_columns = [col for col in df_processed.columns if col != target_column]
df_processed = df_processed[features_columns + [target_column]]

In [9]:
# --- Sauvegarde des données prétraitées ---
df_processed.to_csv('../data/processed/passengers_casatramway_processed.csv', index=False)


In [10]:
import joblib
# --- Sauvegarde du ColumnTransformer (très important pour le déploiement !) ---
joblib.dump(preprocessor, '../saved_models/preprocessor.pkl')


['../saved_models/preprocessor.pkl']

In [11]:
# --- Affichage d'un aperçu des données prétraitées ---
print("Aperçu des 5 premières lignes des données prétraitées :")
print(df_processed.head())
print("\nInformations sur les types de données des données prétraitées :")
print(df_processed.info())

Aperçu des 5 premières lignes des données prétraitées :
   Jour_Semaine_Friday  Jour_Semaine_Monday  Jour_Semaine_Saturday  \
0                  0.0                  0.0                    1.0   
1                  0.0                  0.0                    0.0   
2                  0.0                  1.0                    0.0   
3                  0.0                  0.0                    0.0   
4                  0.0                  0.0                    0.0   

   Jour_Semaine_Sunday  Jour_Semaine_Thursday  Jour_Semaine_Tuesday  \
0                  0.0                    0.0                   0.0   
1                  1.0                    0.0                   0.0   
2                  0.0                    0.0                   0.0   
3                  0.0                    0.0                   1.0   
4                  0.0                    0.0                   0.0   

   Jour_Semaine_Wednesday  Mois   Annee  Est_Week_End  ...  \
0                     0.0   1.0  2